In [1]:
import scipy
from scipy.stats import norm
from math import exp, expm1,sqrt, log
import math



# Question 1

## a)

In [2]:
def Utility_k(a,x1,x2):
    uk = a -0.003*x1 -0.04*x2
    return uk

In [3]:
def Bi_logit_m(U_bus,U_auto):
    prob_bus = exp(U_bus)/( exp(U_bus)+exp(U_auto) ) 
    prob_auto = 1- prob_bus
    return prob_bus, prob_auto

In [4]:
U_auto = Utility_k(-0.2,120,30)
U_bus = Utility_k(-0.6,30,55)

In [5]:
U_bus, U_auto

(-2.89, -1.76)

In [6]:
portion = Bi_logit_m(U_bus,U_auto)
print ('portions of bus and auto is ' + str(portion) +'respectively')

portions of bus and auto is (0.24416110090520288, 0.7558388990947971)respectively


In [7]:
OD_trip = 10000

In [8]:
print ('revenue equals to ' 
       +str(round(OD_trip *portion[0]*30/100))  + ' US dollar') 


revenue equals to 732 US dollar


## b)
$ P _ { i } ( 1 ) = \Phi \left( \frac { \beta _ { 1 } \mathbf { X } _ { \mathrm { il } } - \beta _ { 2 } \mathbf { X } _ { \mathrm { i2 } } } { \sqrt { \sigma _ { 1 } ^ { 2 } + \sigma _ { 2 } ^ { 2 } - 2 \sigma _ { 12 } } } \right) = \Phi ( z )   $  

$P ( A ) = \Phi \left( \frac { U _ { A } - U _ { B } } { \sqrt { \sigma _ { A } ^ { 2 } + \sigma _ { B } ^ { 2 } - 2 \rho \sigma _ { A } \sigma _ { B } } } \right)$

In [9]:
def bi_pro_M (U1, U2, sig1, sig2, rho):
    '''
    coefficient correlation (rho)
    '''
    numerator = U1- U2
    denominator = sqrt(sig1**2 + sig2**2 - 2* rho*sig1*sig2 )
    Z_value = numerator/denominator 
    P1 = norm.cdf(Z_value)
    return P1

In [10]:
bi_pro_M(U_bus,U_auto, 0.8,1.2,0.3)

0.17841731684999523

In [11]:
print ('The revenue of the local bus is ' 
       + str(OD_trip * round(bi_pro_M(U_bus,U_auto, 0.8,1.2,0.3),2)* 30/100 ) 
      + ' US dollar')

The revenue of the local bus is 540.0 US dollar


## c) 

In [12]:
U_BRT = Utility_k(-0.40,60,45)

In [13]:
U_BRT,U_auto,U_bus

(-2.38, -1.76, -2.89)

In [14]:
def Mul_logit_m(*argv):
    sum_U = 0
    for i in range(len(argv)): 
        sum_U = sum_U +  exp(argv[i])
    
    ls= [[]]* len(argv)
    for i in range(len(argv)): 
        ls[i] = exp(argv[i]) /sum_U
        
    return ls

In [15]:
prob3 = Mul_logit_m(U_BRT,U_auto,U_bus)

In [16]:
print ("The probabilty of take BRT, auto and bus are {}%, {}% and {}% respectively" 
       .format(round(prob3[0]*100,1), 
               round(prob3[1]*100,1), 
               round(prob3[2]*100,1))
      )

The probabilty of take BRT, auto and bus are 28.9%, 53.7% and 17.4% respectively


In [17]:
print ('The revenue is ${}'
       .format(int(OD_trip)* (0.6 * round(prob3[0]*100,1)+ 0.3 *round(prob3[2]*100,1)) ) )

The revenue is $225600.0


# Problem 2

After implementing the BRT service, the public transportation authority did a survey and found that the actual percentage of trips using public transit service is lower than expected (calculated in Question 1). They undertook a nested logit model estimation study and found that $U_T =a+ \theta * Logsum $  with $a=-0.42 and \theta=1.1 $. Please determine the corresponding mode shares based on the nested logit model.

### netsed level 

In [18]:
def Utility_T(a,theta, U_BRT,U_bus):
    logsum = log(exp(U_BRT)+ exp(U_bus))    #nature log
    ut = a + logsum* theta 
    return ut

In [19]:
a = -0.42 
theta = 1.1 

In [20]:
Utility_T(a,theta, U_BRT,U_bus)

-2.5206553501506375

In [21]:
U_transit =Utility_T(a,theta, U_BRT,U_bus)

In [22]:
## primary model
Bi_logit_m(U_transit,U_auto)

(0.3185039995137753, 0.6814960004862247)

In [23]:
transit_share = Bi_logit_m(U_transit,U_auto)[0]

In [24]:
##
def nested_logit_NL(U_transit,U_bus,U_auto):
    P_b_t =  Bi_logit_m(U_bus,U_BRT)[0] *transit_share
    P_BRt_t =  Bi_logit_m(U_BRT,U_bus)[0] *transit_share
    return P_b_t, P_BRt_t
nested_logit_NL(U_transit,U_bus,U_auto)

(0.11950063847347901, 0.19900336104029628)

In [25]:
prob4 = nested_logit_NL(U_transit,U_bus,U_auto)

In [26]:
print ('The revenue is ${}'
       .format(int(OD_trip)* (0.6 * round(prob4[1],2)+ 
                              0.3 *round(prob4[0],2)) ) )

The revenue is $1560.0


# Question 3 
On weekends, 1000 vehicle-based shopping trips leave the residential area for the two shopping centers. Trips can be made either by auto or by bus. A joint shopping- trip mode-destination choice logit model is estimated with the following coefficients: 
...


If shopping center 1 has 250,000ft 2 of commercial floor space and shopping center 2 has 400,000ft 2 , please determine the distribution of the weekend shopping trips by destination and mode. 

In [27]:
Var = [[0.6,-0.3,0.012],
       [0,-0.3,0.012]]     ## coefficent auto; bus

In [28]:
tt_dest = [[1,8,250],
           [1,15,400],
          [1,14,250],
          [1,22,400]]  ## auto ;bus

In [29]:
U_auto_D1=sum(x*y for x,y in zip(Var[0],tt_dest[0]))  
U_auto_D2=sum(x*y for x,y in zip(Var[0],tt_dest[1])) 
U_bus_D1=sum(x*y for x,y in zip(Var[1],tt_dest[2]))  
U_bus_D2=sum(x*y for x,y in zip(Var[1],tt_dest[3])) 

In [30]:
Mul_logit_m(U_auto_D1,U_auto_D2,U_bus_D1,U_bus_D2)

[0.5315407674210888,
 0.39377508554068574,
 0.048220290510324795,
 0.026463856527900616]

In [31]:
prob5 = Mul_logit_m(U_auto_D1,U_auto_D2,U_bus_D1,U_bus_D2)

In [32]:
print (
        'The number of auto trip to shopping center 1 is {}'
       .format(round(prob5[0],2)*1000) + '\n'
        'The number of auto trip to shopping center 2 is {}'
       .format(round(prob5[1],2)*1000) + '\n'
        'The number of bus trip to shopping center 1 is {}'
       .format(round(prob5[2],2)*1000) + '\n'
        'The number of bus trip to shopping center 2 is {}'
       .format(round(prob5[3],2)*1000) 
              
        )

The number of auto trip to shopping center 1 is 530.0
The number of auto trip to shopping center 2 is 390.0
The number of bus trip to shopping center 1 is 50.0
The number of bus trip to shopping center 2 is 30.0


# Question4

$
P_{A}= e^{U_A} / (e^{U_A}+e^{U_B})\\
= 1/(1+ e^{U_B - U_A} ) =0.35
$


In [55]:
# before improvement on bus ridership

In [43]:
coeff = -0.025
bus_s_1 = 0.65
delta_A = coeff * 1 
delta_B = coeff * (25-30)

$
P_{A}'= e^{U_A'} / (e^{U_A'}+e^{U_B'})\\
= 1/(1+ e^{U_B - U_A + \Delta U_B - \Delta U_A} )
$


In [67]:
log(1/0.35 - 1)

0.6190392084062235

In [63]:
delta_B - delta_A

0.15

In [70]:
# after P_a2
P_a2= 1/(1+exp(log(1/0.35 - 1) + delta_B - delta_A)  )

In [71]:
P_a2

0.3166869814172787

In [73]:
print (
        'After the improvement the percentage of auto trip is  {}%'
       .format(round(P_a2,2)*100) + '\n'
        'The percentage of bus trip  is {}%'
       .format(round(1-P_a2,2)*100) + '\n'
    ) 

After the improvement the percentage of auto trip is  32.0%
The percentage of bus trip  is 68.0%

